In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def format_data(stock_data_file, report_data_file_1, report_data_file_2):
    #P
    stock_data = pd.read_csv(f'stock data/{stock_data_file}')

    # Chuyển cột 'date' thành datetime
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True)

    # Tính quý tương ứng
    stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')

    # Tính trung bình giá đóng cửa theo quý
    quarterly_avg = stock_data.groupby('quarter')['Close'].mean().reset_index()

    # Tạo danh sách quý từ 2019Q1 đến 2024Q4
    all_quarters = pd.period_range('2019Q1', '2024Q4', freq='Q')
    all_quarters_df = pd.DataFrame({'quarter': all_quarters})

    # Merge với dữ liệu thực tế, đảm bảo đủ các quý
    merged = pd.merge(all_quarters_df, quarterly_avg, on='quarter', how='left')

    # Đổi tên cột 'close' thành 'P'
    merged.rename(columns={'Close': 'P'}, inplace=True)
    
    # Đổi định dạng cột 'quarter'
    merged['quarter'] = merged['quarter'].astype(str)  # Chuyển cột 'quarter' thành chuỗi
    merged['quarter'] = merged['quarter'].apply(convert_quarter)  # Áp dụng hàm convert_quarter

    # Chọn cả cột 'quarter' và 'P' vào một DataFrame
    merged_quarterly = merged[['quarter', 'P']]

    # Chuyển DataFrame thành Series
    P = merged_quarterly.set_index('quarter')['P']

    # D/E
    report_data_1 = pd.read_csv(f'report 1/{report_data_file_1}', index_col=0)
    
    # Đổi tên cột
    report_data_1.columns = [convert_quarter(c) for c in report_data_1.columns]

    # Lấy feature thứ 65 và 95 (index 64 và 94)
    de_numerator = report_data_1.iloc[65].copy()
    de_denominator = report_data_1.iloc[95].copy()

    de_numerator_clean = clean_numeric(de_numerator)
    de_denominator_clean = clean_numeric(de_denominator)

    # Đồng bộ index và tính D/E
    common_quarters = de_numerator_clean.index.intersection(de_denominator_clean.index)
    de_ratio = de_numerator_clean[common_quarters] / de_denominator_clean[common_quarters]

    # Tạo bảng kết quả
    DE = de_ratio.rename('D/E')
    DE.index.name = 'quarter'

    report_data_2 = pd.read_csv(f'report 2/{report_data_file_2}', index_col=0)

    report_data_2.columns = [convert_quarter(c) for c in report_data_2.columns]

    line_19_fpt_2 = report_data_2.iloc[19]

    # Lấy dòng 99 từ file Tổng hợp báo cáo FPT (index 98)
    line_99_fpt = report_data_1.iloc[98]

    line_19_fpt_2_clean = clean_numeric(line_19_fpt_2)

    line_99_fpt_clean = clean_numeric(line_99_fpt)

    EPS = line_19_fpt_2_clean[common_quarters] / line_99_fpt_clean[common_quarters]

    EPS.index.name = 'quarter'

    EPS.name = 'EPS'

    common_quarters_2 = EPS.index.intersection(P.index)

    PE = P[common_quarters_2] / EPS[common_quarters_2]

    PE.name = 'P/E'

    de_size = report_data_1.iloc[63].copy()
    de_size_clean = clean_numeric(de_size)
    SIZE = np.log(de_size_clean).rename('SIZE')
    SIZE.index.name = 'quarter'

    result = pd.concat([P, EPS, DE, PE, SIZE], axis=1)

    return result
    
# Loại bỏ dấu phẩy và chuyển về số
def clean_numeric(series):
    return pd.to_numeric(series.astype(str).str.replace(',', '', regex=False), errors='coerce')

# Chuẩn hóa tên cột từ "Quý ? năm ?" → "YYYYQ#"
def convert_quarter(col):
    match = re.match(r'Quý (\d+) năm (\d{4})', col)
    if match:
        q, y = match.groups()
        return f'{y}Q{q}'
    return col

In [3]:
companies = {
    # Nông nghiệp
    "Hoàng Anh Gia Lai": "HAG",
    "Tập đoàn PAN": "PAN",
    "Dabaco Việt Nam": "DBC",
    "Giống cây trồng Trung Ương": "NSC",
    "Mía đường Lam Sơn": "LSS",
    "Mía đường Thành Thành Công Tây Ninh": "SBT",
    "Mía đường Sơn La": "SLS",
    "Mía đường Sóc Trăng": "SLS",
    "Thực phẩm Sao Ta": "FMC",
    "Nông nghiệp Quốc tế Hoàng Anh Gia Lai": "HNG",
    "Thực phẩm Cholimex": "CMF",
    "Giống cây trồng Miền Nam": "SSC",
    "Công ty cổ phần Vĩnh Hoàn": "VHC",
    "Công ty cổ phần Nam Việt": "ANV",
    "Công ty cổ phần Thủy sản Minh Phú": "MPC",
    "Công ty cổ phần Thực phẩm Bích Chi": "BFC",
    "Công ty cổ phần Tập đoàn Lộc Trời": "LTG",
    "CTCP Nông nghiệp BaF Việt Nam": "BAF",
    "CTCP Dược phẩm Cửu Long": "DCL",
    "CTCP Thuốc sát trùng Việt Nam": "VFG",
    "CTCP XNK Thủy sản Bến Tre": "ABT",
    "CTCP XNK An Giang": "AGM",
    "CTCP Chế biến Gỗ Đức Thành": "GDT",
    "CTCP Tập đoàn Dược Bảo Châu": "BCP",
    "CTCP Tập đoàn Sao Mai": "ASM",
    "CTCP Dầu thực vật Tường An": "TAC",
    "CTCP Dầu thực vật Cái Lân": "VOC"
}

for company_name, company_code in companies.items():
    stock_data_file = f"{company_name}_stock_data.csv"
    report_data_file_1 = f"Tổng hợp báo cáo {company_name}.csv"
    report_data_file_2 = f"Tổng hợp báo cáo {company_name} 2.csv"
    
    # Gọi format_data và lưu kết quả vào file CSV
    result = format_data(stock_data_file, report_data_file_1, report_data_file_2)
    result.to_csv(f"pre-processing/{company_name}_data.csv")


C:\Users\hype1\AppData\Local\Temp\ipykernel_14180\2490079865.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_14180\2490079865.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_14180\2490079865.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_14180\2490079865.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_data['quarter'] = stock_data['Date'].dt.to_period('Q')
C:\Users\hype1\AppData\Local\Temp\ipykernel_14180\2490079865.py:9: UserWarning: Converting to PeriodArray/Index 